In [125]:
# !pip install geemap 
# !pip install geopandas
# !pip install mapclassify
# !pip install contextily
# !pip install matplotlib-scalebar

In [126]:
# import ee
# import geemap
# import geopandas as gpd
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap
# from matplotlib.colors import Normalize
# from matplotlib_scalebar.scalebar import ScaleBar
# import contextily as cx

In [127]:
# ee.Authenticate()
# ee.Initialize()

In [128]:
# Load UNOSAT classifications
crop17 = ee.Image("users/kpalmkron/Agriculture_Syria_2016_2017");
crop18 = ee.Image("users/kpalmkron/Agriculture_Syria_2017_2018");
crop19 = ee.Image("users/kpalmkron/Agriculture_Syria_2018_2019");
crop20 = ee.Image("users/kpalmkron/Agriculture_Syria_2019_2020");
crop21 = ee.Image("users/kpalmkron/Agriculture_Syria_2020_2021");

Load and covert AOIs

In [129]:
# Load AOIs (community polygons
Raweished = ee.FeatureCollection("users/reachsyriagee/ABA/Raweished").map(lambda f:f.set('name', 'Raweished_north')); # Northern polygon only
Sahlabiyeh = ee.FeatureCollection("users/reachsyriagee/ABA/Sahlabiyeh").map(lambda f:f.set('name', 'Sahlabiyeh'));
Tal_Brak = ee.FeatureCollection("users/reachsyriagee/ABA/Tal_brak").map(lambda f:f.set('name', 'Tal_Brak'));
Qabasin = ee.FeatureCollection("users/reachsyriagee/ABA/Qabasin").map(lambda f:f.set('name', 'Qabasin'));
Jurneyyeh = ee.FeatureCollection("users/reachsyriagee/ABA/Jurneyyeh").map(lambda f:f.set('name', 'Jurneyyeh'));
Jarablus = ee.FeatureCollection("users/reachsyriagee/ABA/Jarablus").map(lambda f:f.set('name', 'Jarablus'));
Darkosh = ee.FeatureCollection("users/reachsyriagee/ABA/Darkosh").map(lambda f:f.set('name', 'Darkosh'));
Akhtrein = ee.FeatureCollection("users/reachsyriagee/ABA/Akhtrein").map(lambda f:f.set('name', 'Akhtrein'));

# Combining AOIs to one fC
all_ABAs = Sahlabiyeh.merge(Tal_Brak).merge(Raweished).merge(Qabasin).merge(Jurneyyeh).merge(Jarablus).merge(Darkosh).merge(Akhtrein)

# Convert fC to pandas df
all_ABAs_df = geemap.ee_to_pandas(all_ABAs)


Prepare AOIs for visualization

In [130]:
# Paint fCs onto images
def fc_to_img(fc):
    empty = ee.Image().byte();
    img = empty.paint(
      featureCollection = fc,
      color = 1,
      width = 3)
    return img

Raweished_north_img = fc_to_img(Raweished)
all_ABAs_img = fc_to_img(all_ABAs)
Sahlabiyeh_img = fc_to_img(Sahlabiyeh)
Tal_Brak_img = fc_to_img(Tal_Brak)
Raweished_img = fc_to_img(Raweished)
Qabasin_img = fc_to_img(Qabasin)
Jurneyyeh_img = fc_to_img(Jurneyyeh)
Jarablus_img = fc_to_img(Jarablus)
Darkosh_img = fc_to_img(Darkosh)
Akhtrein_img = fc_to_img(Akhtrein)

Visualize AOIs and cropland maps

In [131]:
imageVisParam = {"opacity":1,"bands":["b1"],"max":1,"palette":["ffffff","168103"]}

Map = geemap.Map(center=(35.8, 39.8), zoom=8, lite_mode=False) 
Map.add_basemap('SATELLITE')

Map.addLayer(ee.Image(crop17), imageVisParam, 'crop17')
Map.addLayer(ee.Image(crop18), imageVisParam, 'crop18')
Map.addLayer(ee.Image(crop19), imageVisParam, 'crop19')
Map.addLayer(ee.Image(crop20), imageVisParam, 'crop20')
Map.addLayer(ee.Image(crop21), imageVisParam, 'crop21')

Map.addLayer(all_ABAs_img, '', 'all_ABAs_img')
# Map.addLayer(Sahlabiyeh_img, '', 'Sahlabiyeh')
# Map.addLayer(Sahlabiyeh_shared_img, '', 'Sahlabiyeh_shared')
# Map.addLayer(Tal_Brak_img, '', 'Tal_Brak')
# Map.addLayer(Qabasin_img, '', 'Qabasin')
# Map.addLayer(Jurneyyeh_img, '', 'Jurneyyeh')
# Map.addLayer(Jarablus_img, '', 'Jarablus')
# Map.addLayer(Darkosh_img, '', 'Darkosh')
# Map.addLayer(Akhtrein_img, '', 'Akhtrein')
# Map.addLayer(Raweished_img, '', 'Raweished')



In [132]:
Map

Map(center=[35.8, 39.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

Extracting Cropland Area

In [133]:
# Function to extract cropland area per admin unit
def extractor(adm, lc):

  # Remove non-crop pixels
  only_crop = lc.gt(0)
  
  # Reduce to cropland pixel count per admin unit
  cropland_px = only_crop.reduceRegions(
                      collection = adm,
                      reducer = ee.Reducer.sum(), # 'sum' is the same as 'count' of all cropland pixels since crop pixels values are 1 
                      scale = 10).map(lambda f: f.set('px',f.get('sum'))) # change property name to sum

  # Convert px count to ha
  cropland_ha = cropland_px.map(lambda f: f.set('crop_ha', ee.Number(f.get('sum')).multiply(0.01)))

  # Calculate total pixel count (crop and non-crop) as a reference check
  cropland_ha = lc.reduceRegions(
                      collection = cropland_ha,
                      reducer = ee.Reducer.count(), 
                      scale = 10)

#  cropland_ha.map(lambda f: f.set('count', ee.Reducer.count()))
  cropland_ha = cropland_ha.map(lambda f: f.set('total_ha', ee.Number(f.get('count')).multiply(0.01))) # Calculating number of pixels (crop and non-crop) and converting to ha

  # Drop geometries
  cropland_ha = cropland_ha.map(lambda f: f.setGeometry())
  
  return cropland_ha

In [134]:
# Produce featureCollection with all attributes
crop17_area = extractor(all_ABAs, crop17)
crop18_area = extractor(all_ABAs, crop18)
crop19_area = extractor(all_ABAs, crop19)
crop20_area = extractor(all_ABAs, crop20)
crop21_area = extractor(all_ABAs, crop21)

# Converting featureCollection to pandas dataframes
crop17 = geemap.ee_to_pandas(crop17_area)
crop18 = geemap.ee_to_pandas(crop18_area)
crop19 = geemap.ee_to_pandas(crop19_area)
crop20 = geemap.ee_to_pandas(crop20_area)
crop21 = geemap.ee_to_pandas(crop21_area)

Analysing area and changes

In [135]:
# Combined dataframes
comb = crop17.rename(columns={"total_ha": "ha17"}).drop(['px'], axis = 1)

comb['total_ha'] = crop17['total_ha']    # This is the total area within the polygon, include agriculture and non-agriculture, in hectares
comb['crop_ha_17'] = crop17['crop_ha']
comb['crop_ha_18'] = crop18['crop_ha']
comb['crop_ha_19'] = crop19['crop_ha']
comb['crop_ha_20'] = crop20['crop_ha']
comb['crop_ha_21'] = crop21['crop_ha']

# Clean up the columns
comb = comb.drop('crop_ha', 1)
comb = comb.drop('sum', 1)
comb = comb.drop('Shape_Leng', 1)
comb = comb.drop('Shape_Area', 1)
comb = comb.drop('ha17', 1)
comb = comb.drop('count', 1)
comb = comb.drop('OBJECTID', 1)

comb

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of Dat

,name,total_ha,crop_ha_17,crop_ha_18,crop_ha_19,crop_ha_20,crop_ha_21
0,Sahlabiyeh,2698.34,1837.904902,1904.339412,1646.251490,1623.023765,1779.383059
1,Tal_Brak,316.69,58.899569,20.487412,103.111569,117.563373,9.788078
2,Raweished_north,22547.67,1957.318941,2266.537961,6727.736314,3328.813451,2531.109412
3,Qabasin,4549.25,3774.535647,2619.978431,3393.256627,3606.229569,2166.263804
4,Jurneyyeh,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Jurneyyeh,752.56,54.448275,49.168549,174.866784,89.370980,51.865843
6,Jarablus,1255.28,599.106157,519.910353,470.808431,508.574706,528.020118
7,Darkosh,651.56,358.243059,320.923020,274.984039,296.567608,285.741216
8,Akhtrein,7048.21,6345.939412,5821.750275,5252.243059,6217.166627,4951.964314


Extracting stats

In [136]:
# Aggregating across district names (in case there are multiple polygones for one district)
comb_stats = comb.groupby(['name']).sum()

# Rounding off to one decimal point
comb_stats = comb_stats.round(1)

comb_stats = comb_stats.drop('Darkosh')

comb_stats

,total_ha,crop_ha_17,crop_ha_18,crop_ha_19,crop_ha_20,crop_ha_21
name,,,,,,
Akhtrein,7048.2,6345.9,5821.8,5252.2,6217.2,4952.0
Jarablus,1255.3,599.1,519.9,470.8,508.6,528.0
Jurneyyeh,752.6,54.4,49.2,174.9,89.4,51.9
Qabasin,4549.2,3774.5,2620.0,3393.3,3606.2,2166.3
Raweished_north,22547.7,1957.3,2266.5,6727.7,3328.8,2531.1
Sahlabiyeh,2698.3,1837.9,1904.3,1646.3,1623.0,1779.4
Tal_Brak,316.7,58.9,20.5,103.1,117.6,9.8


In [137]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [138]:
comb_stats.round(1).to_excel("all_crop_ABA_v4.xlsx")
!cp all_crop_ABA_v4.xlsx "drive/My Drive/"